# 論文 7：使用深度卷積神經網路進行 ImageNet 分類
## Alex Krizhevsky, Ilya Sutskever, Geoffrey E. Hinton (2012)

### AlexNet：引發深度學習革命的 CNN

AlexNet 以 15.3% 的 top-5 錯誤率贏得 ImageNet 2012，大幅領先第二名（26.2%）。這篇論文重新點燃了對深度學習的興趣。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import correlate2d

np.random.seed(42)

## 卷積層實作

CNN 的核心建構元件

In [ ]:
def relu(x):
    return np.maximum(0, x)

def conv2d(input_image, kernel, stride=1, padding=0):
    """
    2D 卷積運算
    
    input_image: (H, W) 或 (C, H, W)
    kernel: (out_channels, in_channels, kH, kW)
    """
    if len(input_image.shape) == 2:
        input_image = input_image[np.newaxis, :, :]
    
    in_channels, H, W = input_image.shape
    out_channels, _, kH, kW = kernel.shape
    
    # 添加填充
    if padding > 0:
        input_padded = np.pad(input_image, 
                             ((0, 0), (padding, padding), (padding, padding)), 
                             mode='constant')
    else:
        input_padded = input_image
    
    # 輸出維度
    out_H = (H + 2*padding - kH) // stride + 1
    out_W = (W + 2*padding - kW) // stride + 1
    
    output = np.zeros((out_channels, out_H, out_W))
    
    # 執行卷積
    for oc in range(out_channels):
        for i in range(out_H):
            for j in range(out_W):
                h_start = i * stride
                w_start = j * stride
                
                # 提取區塊
                patch = input_padded[:, h_start:h_start+kH, w_start:w_start+kW]
                
                # 與核進行卷積
                output[oc, i, j] = np.sum(patch * kernel[oc])
    
    return output

def max_pool2d(input_image, pool_size=2, stride=2):
    """
    最大池化運算
    """
    C, H, W = input_image.shape
    
    out_H = (H - pool_size) // stride + 1
    out_W = (W - pool_size) // stride + 1
    
    output = np.zeros((C, out_H, out_W))
    
    for c in range(C):
        for i in range(out_H):
            for j in range(out_W):
                h_start = i * stride
                w_start = j * stride
                
                pool_region = input_image[c, h_start:h_start+pool_size, 
                                         w_start:w_start+pool_size]
                output[c, i, j] = np.max(pool_region)
    
    return output

# 測試卷積
test_image = np.random.randn(1, 8, 8)
test_kernel = np.random.randn(3, 1, 3, 3) * 0.1

conv_output = conv2d(test_image, test_kernel, stride=1, padding=1)
print(f"輸入形狀：{test_image.shape}")
print(f"核形狀：{test_kernel.shape}")
print(f"卷積輸出形狀：{conv_output.shape}")

pooled = max_pool2d(conv_output, pool_size=2, stride=2)
print(f"最大池化後：{pooled.shape}")

## AlexNet 架構（簡化版）

原始版：227x227x3 → 5 層卷積 → 3 層全連接 → 1000 類別

我們簡化版用於 32x32 圖像

In [ ]:
class AlexNetSimplified:
    def __init__(self, num_classes=10):
        """
        用於 32x32 圖像的簡化 AlexNet（如 CIFAR-10）
        
        架構：
        - Conv1: 3x3x3 -> 32 個濾波器
        - MaxPool
        - Conv2: 32 -> 64 個濾波器
        - MaxPool
        - Conv3: 64 -> 128 個濾波器
        - 全連接層
        """
        # 卷積層
        self.conv1_filters = np.random.randn(32, 3, 3, 3) * 0.01
        self.conv1_bias = np.zeros(32)
        
        self.conv2_filters = np.random.randn(64, 32, 3, 3) * 0.01
        self.conv2_bias = np.zeros(64)
        
        self.conv3_filters = np.random.randn(128, 64, 3, 3) * 0.01
        self.conv3_bias = np.zeros(128)
        
        # 全連接層（卷積後：128 * 4 * 4 = 2048）
        self.fc1_weights = np.random.randn(2048, 512) * 0.01
        self.fc1_bias = np.zeros(512)
        
        self.fc2_weights = np.random.randn(512, num_classes) * 0.01
        self.fc2_bias = np.zeros(num_classes)
    
    def forward(self, x, use_dropout=False, dropout_rate=0.5):
        """
        前向傳遞
        x: (3, 32, 32) 圖像
        """
        # Conv1 + ReLU + MaxPool
        conv1 = conv2d(x, self.conv1_filters, stride=1, padding=1)
        conv1 += self.conv1_bias[:, np.newaxis, np.newaxis]
        conv1 = relu(conv1)
        pool1 = max_pool2d(conv1, pool_size=2, stride=2)  # 32 x 16 x 16
        
        # Conv2 + ReLU + MaxPool
        conv2 = conv2d(pool1, self.conv2_filters, stride=1, padding=1)
        conv2 += self.conv2_bias[:, np.newaxis, np.newaxis]
        conv2 = relu(conv2)
        pool2 = max_pool2d(conv2, pool_size=2, stride=2)  # 64 x 8 x 8
        
        # Conv3 + ReLU + MaxPool
        conv3 = conv2d(pool2, self.conv3_filters, stride=1, padding=1)
        conv3 += self.conv3_bias[:, np.newaxis, np.newaxis]
        conv3 = relu(conv3)
        pool3 = max_pool2d(conv3, pool_size=2, stride=2)  # 128 x 4 x 4
        
        # 展平
        flattened = pool3.reshape(-1)
        
        # FC1 + ReLU + Dropout
        fc1 = np.dot(flattened, self.fc1_weights) + self.fc1_bias
        fc1 = relu(fc1)
        
        if use_dropout:
            dropout_mask = (np.random.rand(*fc1.shape) > dropout_rate).astype(float)
            fc1 = fc1 * dropout_mask / (1 - dropout_rate)
        
        # FC2（輸出）
        output = np.dot(fc1, self.fc2_weights) + self.fc2_bias
        
        return output

# 建立模型
alexnet = AlexNetSimplified(num_classes=10)
print("AlexNet（簡化版）已建立")

# 測試前向傳遞
test_img = np.random.randn(3, 32, 32)
output = alexnet.forward(test_img)
print(f"輸入：(3, 32, 32)")
print(f"輸出：{output.shape}（類別分數）")

## 生成合成圖像資料

In [ ]:
def generate_simple_images(num_samples=100, image_size=32):
    """
    生成具有不同模式的簡單合成圖像
    類別：
    0: 水平條紋
    1: 垂直條紋
    2: 對角條紋
    3: 棋盤格
    4: 圓形
    5: 方形
    6: 十字
    7: 三角形
    8: 隨機雜訊
    9: 純色
    """
    X = []
    y = []
    
    for i in range(num_samples):
        class_label = i % 10
        img = np.zeros((3, image_size, image_size))
        
        if class_label == 0:  # 水平條紋
            for row in range(0, image_size, 4):
                img[:, row:row+2, :] = 1
        
        elif class_label == 1:  # 垂直條紋
            for col in range(0, image_size, 4):
                img[:, :, col:col+2] = 1
        
        elif class_label == 2:  # 對角線
            for i in range(image_size):
                if i < image_size:
                    img[:, i, i] = 1
                    if i+1 < image_size:
                        img[:, i, i+1] = 1
        
        elif class_label == 3:  # 棋盤格
            for i in range(0, image_size, 4):
                for j in range(0, image_size, 4):
                    if (i//4 + j//4) % 2 == 0:
                        img[:, i:i+4, j:j+4] = 1
        
        elif class_label == 4:  # 圓形
            center = image_size // 2
            radius = image_size // 3
            y_grid, x_grid = np.ogrid[:image_size, :image_size]
            mask = (x_grid - center)**2 + (y_grid - center)**2 <= radius**2
            img[:, mask] = 1
        
        elif class_label == 5:  # 方形
            margin = image_size // 4
            img[:, margin:-margin, margin:-margin] = 1
        
        elif class_label == 6:  # 十字
            mid = image_size // 2
            thickness = 3
            img[:, mid-thickness:mid+thickness, :] = 1
            img[:, :, mid-thickness:mid+thickness] = 1
        
        elif class_label == 7:  # 三角形
            for i in range(image_size):
                width = int((i / image_size) * image_size / 2)
                start = image_size // 2 - width
                end = image_size // 2 + width
                img[:, i, start:end] = 1
        
        elif class_label == 8:  # 隨機雜訊
            img = np.random.rand(3, image_size, image_size)
        
        else:  # 純色
            img[:] = 0.7
        
        # 添加顏色變化
        color = np.random.rand(3, 1, 1)
        img = img * color
        
        # 添加雜訊
        img += np.random.randn(3, image_size, image_size) * 0.1
        img = np.clip(img, 0, 1)
        
        X.append(img)
        y.append(class_label)
    
    return np.array(X), np.array(y)

# 生成資料集
X_train, y_train = generate_simple_images(200)
X_test, y_test = generate_simple_images(50)

print(f"訓練集：{X_train.shape}")
print(f"測試集：{X_test.shape}")

# 視覺化樣本
class_names = ['水平條紋', '垂直條紋', '對角線', '棋盤格', '圓形', 
               '方形', '十字', '三角形', '雜訊', '純色']

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    # 找到每個類別的第一個出現
    idx = np.where(y_train == i)[0][0]
    img = X_train[idx].transpose(1, 2, 0)  # CHW -> HWC
    axes[i].imshow(img)
    axes[i].set_title(class_names[i])
    axes[i].axis('off')

plt.suptitle('合成圖像資料集（10 個類別）', fontsize=14)
plt.tight_layout()
plt.show()

## 資料增強

AlexNet 廣泛使用資料增強 - 這是一項關鍵創新

In [ ]:
def random_flip(img):
    """水平翻轉"""
    if np.random.rand() > 0.5:
        return img[:, :, ::-1].copy()
    return img

def random_crop(img, crop_size=28):
    """隨機裁剪"""
    _, h, w = img.shape
    top = np.random.randint(0, h - crop_size + 1)
    left = np.random.randint(0, w - crop_size + 1)
    
    cropped = img[:, top:top+crop_size, left:left+crop_size]
    
    # 調整回原始大小
    # 簡單最近鄰（用於示範）
    scale_h = h / crop_size
    scale_w = w / crop_size
    
    resized = np.zeros_like(img)
    for i in range(h):
        for j in range(w):
            src_i = min(int(i / scale_h), crop_size - 1)
            src_j = min(int(j / scale_w), crop_size - 1)
            resized[:, i, j] = cropped[:, src_i, src_j]
    
    return resized

def add_noise(img, noise_level=0.05):
    """添加高斯雜訊"""
    noise = np.random.randn(*img.shape) * noise_level
    return np.clip(img + noise, 0, 1)

def augment_image(img):
    """應用隨機增強"""
    img = random_flip(img)
    img = random_crop(img)
    img = add_noise(img)
    return img

# 展示增強
original = X_train[0]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes[0, 0].imshow(original.transpose(1, 2, 0))
axes[0, 0].set_title('原始')
axes[0, 0].axis('off')

for i in range(1, 8):
    augmented = augment_image(original.copy())
    row = i // 4
    col = i % 4
    axes[row, col].imshow(augmented.transpose(1, 2, 0))
    axes[row, col].set_title(f'增強 {i}')
    axes[row, col].axis('off')

plt.suptitle('資料增強範例', fontsize=14)
plt.tight_layout()
plt.show()

## 視覺化學習到的濾波器

AlexNet 的洞察之一：視覺化網路學到了什麼

In [ ]:
# 視覺化第一層濾波器
filters = alexnet.conv1_filters  # 形狀：(32, 3, 3, 3)

fig, axes = plt.subplots(4, 8, figsize=(16, 8))
axes = axes.flatten()

for i in range(min(32, len(axes))):
    # 正規化濾波器以供視覺化
    filt = filters[i].transpose(1, 2, 0)  # CHW -> HWC
    filt = (filt - filt.min()) / (filt.max() - filt.min() + 1e-8)
    
    axes[i].imshow(filt)
    axes[i].axis('off')
    axes[i].set_title(f'F{i}', fontsize=8)

plt.suptitle('Conv1 濾波器（32 個濾波器，3x3，RGB）', fontsize=14)
plt.tight_layout()
plt.show()

print("這些濾波器學習檢測邊緣、顏色和簡單模式")

## 特徵圖視覺化

In [ ]:
# 處理一張圖像並視覺化特徵圖
test_image = X_train[4]  # 圓形

# 通過第一個卷積層前向傳遞
conv1_output = conv2d(test_image, alexnet.conv1_filters, stride=1, padding=1)
conv1_output += alexnet.conv1_bias[:, np.newaxis, np.newaxis]
conv1_output = relu(conv1_output)

# 視覺化
fig = plt.figure(figsize=(16, 10))

# 原始圖像
ax = plt.subplot(6, 6, 1)
ax.imshow(test_image.transpose(1, 2, 0))
ax.set_title('輸入圖像', fontsize=10)
ax.axis('off')

# 特徵圖
for i in range(min(32, 35)):
    ax = plt.subplot(6, 6, i+2)
    ax.imshow(conv1_output[i], cmap='viridis')
    ax.set_title(f'圖 {i}', fontsize=8)
    ax.axis('off')

plt.suptitle('Conv1 + ReLU 後的特徵圖', fontsize=14)
plt.tight_layout()
plt.show()

print("不同的特徵圖對圖像中的不同模式有反應")

## 測試分類

In [ ]:
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

# 在幾張圖像上測試
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    idx = i * 5  # 每 5 張取一張
    img = X_test[idx]
    true_label = y_test[idx]
    
    # 前向傳遞
    logits = alexnet.forward(img, use_dropout=False)
    probs = softmax(logits)
    pred_label = np.argmax(probs)
    
    # 顯示
    axes[i].imshow(img.transpose(1, 2, 0))
    axes[i].set_title(f'真實：{class_names[true_label]}\n預測：{class_names[pred_label]}\n信心：{probs[pred_label]:.2f}',
                     fontsize=9)
    axes[i].axis('off')

plt.suptitle('AlexNet 預測（未訓練）', fontsize=14)
plt.tight_layout()
plt.show()

print("注意：模型未訓練，所以預測是隨機的！")
print("訓練需要梯度下降，為了清晰起見我們簡化了。")

## 關鍵要點

### AlexNet 創新（2012）：

1. **ReLU 激活函數**：比 sigmoid/tanh 快得多
   - 對正值無飽和
   - 訓練更快（比 tanh 快 6 倍）

2. **Dropout**：強大的正則化
   - 防止過擬合
   - 在全連接層使用（0.5 比率）

3. **資料增強**：
   - 隨機裁剪和翻轉
   - 顏色抖動
   - 人為增加資料集大小

4. **GPU 訓練**：
   - 使用 2 個 GTX 580 GPU
   - 使深度網路訓練成為可能

5. **局部響應正規化（LRN）**：
   - 特徵圖之間的側向抑制
   - 現在不太常用（批次正規化取代了它）

### 架構：
```
輸入 (227x227x3)
  ↓
Conv1 (96 個濾波器, 11x11, 步幅 4) + ReLU + MaxPool
  ↓
Conv2 (256 個濾波器, 5x5) + ReLU + MaxPool
  ↓
Conv3 (384 個濾波器, 3x3) + ReLU
  ↓
Conv4 (384 個濾波器, 3x3) + ReLU
  ↓
Conv5 (256 個濾波器, 3x3) + ReLU + MaxPool
  ↓
FC6 (4096) + ReLU + Dropout
  ↓
FC7 (4096) + ReLU + Dropout
  ↓
FC8 (1000 類別) + Softmax
```

### 影響：
- **贏得 ImageNet 2012**：15.3% top-5 錯誤率（vs 第二名 26.2%）
- **重新點燃深度學習**：展示深度 + 資料 + 計算有效
- **GPU 革命**：使 GPU 成為深度學習必需品
- **啟發現代 CNN**：VGG、ResNet 等都建立在這些想法上

### 為什麼有效：
1. 深層架構（在 2012 年 8 層算深了！）
2. 大型資料集（120 萬張 ImageNet 圖像）
3. GPU 加速（使訓練可行）
4. 智慧正則化（dropout + 資料增強）
5. ReLU 激活函數（更快訓練）

### 現代觀點：
- AlexNet 現在被認為是「簡單」的
- ResNets 有 100+ 層
- 批次正規化取代了 LRN
- 但核心理念仍然存在：
  - 深層層級特徵
  - 卷積用於空間結構
  - 資料增強
  - 正則化